In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import csv
import seaborn as sns
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay)
import time
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.svm import SVC

In [2]:
class_names = {
    0: "Speed limit (20km/h)", 1: "Speed limit (30km/h)", 2: "Speed limit (50km/h)",
    3: "Speed limit (60km/h)", 4: "Speed limit (70km/h)", 5: "Speed limit (80km/h)",
    6: "End of speed limit (80km/h)", 7: "Speed limit (100km/h)", 8: "Speed limit (120km/h)",
    9: "No passing", 10: "No passing for vehicles over 3.5 metric tons",
    11: "Right-of-way at the next intersection", 12: "Priority road", 13: "Yield",
    14: "Stop", 15: "No vehicles", 16: "Vehicles over 3.5 metric tons prohibited",
    17: "No entry", 18: "General caution", 19: "Dangerous curve to the left",
    20: "Dangerous curve to the right", 21: "Double curve", 22: "Bumpy road",
    23: "Slippery road", 24: "Road narrows on the right", 25: "Road work",
    26: "Traffic signals", 27: "Pedestrians", 28: "Children crossing",
    29: "Bicycles crossing", 30: "Beware of ice/snow", 31: "Wild animals crossing",
    32: "End of all speed and passing limits", 33: "Turn right ahead",
    34: "Turn left ahead", 35: "Ahead only", 36: "Go straight or right",
    37: "Go straight or left", 38: "Keep right", 39: "Keep left",
    40: "Roundabout mandatory", 41: "End of no passing",
    42: "End of no passing for vehicles over 3.5 metric tons"
}

In [3]:
augmented_I_RGB = np.load('augmented_I_RGB.npy')      #Shape: (N, 32, 32, 3)
augmented_I_Gray = np.load('augmented_I_Gray.npy')    #Shape: (N, 32, 32)
augmented_L = np.load('augmented_L.npy')              #shape: (N,)

#Verify shapes
print(f"RGB images: {augmented_I_RGB.shape}")
print(f"Grayscale images: {augmented_I_Gray.shape}")
print(f"Labels: {augmented_L.shape}")

RGB images: (78418, 32, 32, 3)
Grayscale images: (78418, 32, 32)
Labels: (78418,)


In [4]:
def extract_hog_features(image):
    return hog(image, pixels_per_cell=(4, 4), cells_per_block=(2, 2), visualize=False)

X = np.array(Parallel(n_jobs=-1)(delayed(extract_hog_features)(img) for img in augmented_I_Gray))
y = augmented_L  

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42,stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### USING CPU

In [6]:
#https://www.researchgate.net/publication/344399165_Traffic_Sign_Recognition_System_TSRS_SVM_and_Convolutional_Neural_Network
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

os.environ['OMP_NUM_THREADS'] = '16'  #Use all 32 threads
os.environ['MKL_NUM_THREADS'] = '16'
os.environ['OPENBLAS_NUM_THREADS'] = '16'

svm = SVC(
    kernel='rbf',
    C=10,
    gamma='scale',
    class_weight='balanced',
    decision_function_shape='ovr',
    random_state=42,
    verbose=2,               
    cache_size=10000,        #10GB RAM cache
    max_iter=10,             #lim
    tol=1e-3,               
    shrinking=True          
)

print(f"Starting training on {len(X_train_scaled)} samples...")
start_time = time.time()
svm.fit(X_train_scaled, y_train)
training_time = time.time() - start_time
print(f"Training time: {training_time/60:.1f} minutes")
#Predictions
y_train_pred = svm.predict(X_train_scaled)  #For training accuracy
y_test_pred = svm.predict(X_test_scaled)    #For test accuracy

#Calculate our eval metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

print(f"\n=== Results ===")
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Support vectors: {len(svm.support_vectors_)}")

Starting training on 62734 samples...
[LibSVM]

E:\Python\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Training time: 1.3 minutes

=== Results ===
Training Accuracy: 0.8519
Test Accuracy: 0.8472
Precision: 0.8583
Recall: 0.8472
F1-Score: 0.8449
Support vectors: 7637


In [7]:
inference_times = []
for _ in range(10):
    start_inf = time.time()
    _ = svm.predict(X_test_scaled[:1]) 
    inference_times.append(time.time() - start_inf)
avg_inference_time = np.mean(inference_times) * 1000  # in milli-seconds
print(f"Average Inference Time: {avg_inference_time:.2f}ms per sample")

Average Inference Time: 13.28ms per sample


In [10]:
from codecarbon import EmissionsTracker
tracker = EmissionsTracker()
tracker.start()
svm.predict(X_test_scaled[:1])
emissions_svm = tracker.stop()
print(f"Emissions for Random Forest: {emissions_svm} kg CO2")

[codecarbon WARNING @ 16:05:40] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 16:05:40] [setup] RAM Tracking...
[codecarbon INFO @ 16:05:40] [setup] CPU Tracking...
[codecarbon WARNING @ 16:05:41] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 16:05:41] CPU Model on constant consumption mode: AMD Ryzen 9 5900XT 16-Core Processor
[codecarbon WARNING @ 16:05:41] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 16:05:41] [setup] GPU Tracking...
[codecarbon INFO @ 16:05:41] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 16:05:41] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: pynvml
            
[codecarbon INFO @ 16:05:41] >>> Tracker's metad

Emissions for Random Forest: 3.4702098304004362e-06 kg CO2


[codecarbon INFO @ 16:05:57] Energy consumed for RAM : 0.000667 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 16:05:57] Delta energy consumed for CPU with constant : 0.007003 kWh, power : 1680.0 W
[codecarbon INFO @ 16:05:57] Energy consumed for All CPU : 0.056035 kWh
[codecarbon INFO @ 16:05:57] Energy consumed for all GPUs : 0.000188 kWh. Total GPU Power : 5.578749821636295 W
[codecarbon INFO @ 16:05:57] 0.056890 kWh of electricity used since the beginning.
[codecarbon INFO @ 16:05:57] 0.137751 g.CO2eq/s mean an estimation of 4,344.113129226963 kg.CO2eq/year
[codecarbon INFO @ 16:05:59] Energy consumed for RAM : 0.000250 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 16:05:59] Delta energy consumed for CPU with constant : 0.007006 kWh, power : 1680.0 W
[codecarbon INFO @ 16:05:59] Energy consumed for All CPU : 0.021013 kWh
[codecarbon INFO @ 16:05:59] Energy consumed for all GPUs : 0.000069 kWh. Total GPU Power : 5.667943756478628 W
[codecarbon INFO @ 16:05:59] 0.021332 kWh of electricity u